In [ ]:
!pip install sentencepiece

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 35.7 MB/s eta 0:00:00


In [ ]:
!pip install transformers
!pip install datasets
!pip install evaluate
!pip install bert_score

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 82.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.8/199.8 KB 21.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 103.5 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.7/468.7 KB 19.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 KB 13.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 57.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.2/212.2 KB 22.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.9/132.9 KB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.6/264.6 KB 22.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 KB 18.6 MB/s eta 0:00:

In [ ]:
import requests
import json
import torch
import os
from tqdm import tqdm

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

import re
import string
import nltk
from nltk.corpus import stopwords
from datasets import load_metric
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
# read data from the json files
train_df = pd.read_json("train.jsonl", lines = True)
test_df = pd.read_json("validation.jsonl", lines = True)

print("train data shape:",train_df.shape)
print("Test data shape:",test_df.shape)

train_df = train_df[["uuid", "postText", "targetTitle", "targetParagraphs", "spoiler", "provenance", "tags", "spoilerPositions"]]
train_df.rename(columns={"postText":"clickbait", "targetTitle": "title", "targetParagraphs":"document", "spoilerPositions": "positions", "tags":"type"}, inplace = True)

test_df = test_df[["uuid", "postText", "targetTitle", "targetParagraphs", "spoiler", "provenance", "tags", "spoilerPositions"]]
test_df.rename(columns={"postText":"clickbait", "targetTitle": "title", "targetParagraphs":"document", "spoilerPositions":"positions", "tags":"type"}, inplace = True)

# Function to Make values in Spoiler_Type(list to string)
def list_to_string(spoiler_type):
  if spoiler_type[0] == "phrase":
    return 0
  elif spoiler_type[0] == "passage":
    return 1
  elif spoiler_type[0] == "multi":
    return 2

train_df["type"] = train_df["type"].apply(list_to_string)
test_df["type"] = test_df["type"].apply(list_to_string)

train data shape: (3200, 14)
Test data shape: (800, 14)


<ipython-input-54-899c60e8dd6b>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_df.rename(columns={"postText":"clickbait", "targetTitle": "title", "targetParagraphs":"document", "spoilerPositions": "positions", "tags":"type"}, inplace = True)
<ipython-input-54-899c60e8dd6b>:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_df["type"] = train_df["type"].apply(list_to_string)


In [ ]:
# total_df_bkp = total_df.copy()
train_df = train_df[train_df['type']==0]
test_df = test_df[test_df['type']==0]
validation_df = train_df.iloc[1258:]
train_df = train_df.iloc[:1174]


In [ ]:
"""
train_data = [
    {
        "context": "Mistborn is a series of epic fantasy novels written by American author Brandon Sanderson.",
        "qas": [
            {
                "id": "00001",
                "is_impossible": False,
                "question": "Who is the author of the Mistborn series?",
                "answers": [
                    {
                        "text": "Brandon Sanderson",
                        "answer_start": 71,
                    }
                ],
            }
        ],
    }
  ]

"""

train_formatted_data = []

for index, row in train_df.iterrows():
    row["document"].append(row["title"])

    train_formatted_data.append({
        # "human_spoiler":row["provenance"].get("humanSpoiler"),
        "human_spoiler":row["spoiler"][0],
        "context": ' '.join(row["document"]),
        "qas": [
            {
                "id": row["uuid"],
                "is_impossible": False,
                "question": row["clickbait"][0],
                "answers": [
                    {
                        "text": row["spoiler"][0],
                        "answer_start": ' '.join(row["document"]).find(row["spoiler"][0]),
                    }
                ],
            }
        ]
    })


eval_formatted_data = []

for index, row in validation_df.iterrows():
    row["document"].append(row["title"])
    
    eval_formatted_data.append({
        # "human_spoiler":row["provenance"].get("humanSpoiler"),
        "human_spoiler":row["spoiler"][0],
        "context": ' '.join(row["document"]),
        "qas": [
            {
                "id": row["uuid"],
                "is_impossible": False,
                "question": row["clickbait"][0],
                "answers": [
                    {
                        "text": row["spoiler"][0],
                        "answer_start": ' '.join(row["document"]).find(row["spoiler"][0]),
                    }
                ],
            }
        ]
    })

test_formatted_data = []

for index, row in test_df.iterrows():
    row["document"].append(row["title"])
    
    test_formatted_data.append({
        # "human_spoiler":row["provenance"].get("humanSpoiler"),
        "human_spoiler":row["spoiler"][0],
        "context": ' '.join(row["document"]),
        "qas": [
            {
                "id": row["uuid"],
                "is_impossible": False,
                "question": row["clickbait"][0],
                "answers": [
                    {
                        "text": row["spoiler"][0],
                        "answer_start": ' '.join(row["document"]).find(row["spoiler"][0]),
                    }
                ],
            }
        ]
    })


In [ ]:
def read_data(formatted_data):  

  contexts = []
  questions = []
  answers = []
  human_spoilers = []

  for passage in formatted_data:
    context = passage['context']
    human_spoiler = passage["human_spoiler"]
    for qa in passage['qas']:
      question = qa['question']
      for answer in qa['answers']:
        contexts.append(context)
        questions.append(question)
        answers.append(answer)
        human_spoilers.append(human_spoiler)

  return contexts, questions, answers, human_spoilers

In [ ]:
train_contexts, train_questions, train_answers, train_human_spoilers = read_data(train_formatted_data)
valid_contexts, valid_questions, valid_answers, valid_human_spoilers = read_data(eval_formatted_data)
test_contexts, test_questions, test_answers, test_human_spoilers = read_data(test_formatted_data)

In [ ]:

def add_end_idx(answers, contexts):
  for answer, context in zip(answers, contexts):
    gold_text = answer['text']
    start_idx = answer['answer_start']
    end_idx = start_idx + len(gold_text)

    # sometimes squad answers are off by a character or two so we fix this
    if context[start_idx:end_idx] == gold_text:
      answer['answer_end'] = end_idx
    elif context[start_idx-1:end_idx-1] == gold_text:
      answer['answer_start'] = start_idx - 1
      answer['answer_end'] = end_idx - 1     # When the gold label is off by one character
    elif context[start_idx-2:end_idx-2] == gold_text:
      answer['answer_start'] = start_idx - 2
      answer['answer_end'] = end_idx - 2     # When the gold label is off by two characters

add_end_idx(train_answers, train_contexts)
add_end_idx(valid_answers, valid_contexts)
add_end_idx(test_answers, test_contexts)

In [ ]:
from transformers import BertTokenizerFast, RobertaTokenizerFast
from transformers import AutoTokenizer, DebertaV2ForQuestionAnswering, DebertaTokenizerFast, DebertaTokenizer, DebertaV2TokenizerFast

tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased')

train_encodings = tokenizer(train_contexts, train_questions, truncation=True, padding=True)
valid_encodings = tokenizer(valid_contexts, valid_questions, truncation=True, padding=True)
test_encodings = tokenizer(test_contexts, test_questions, truncation=True, padding=True)

In [ ]:
import gc
gc.collect()
torch.cuda.empty_cache()

In [ ]:
torch.cuda.empty_cache()

In [ ]:
train_encodings.keys()

dict_keys(['input_ids', 'token_type_ids', 'attention_mask'])

In [ ]:
no_of_encodings = len(train_encodings['input_ids'])
print(f'We have {no_of_encodings} context-question pairs')

We have 1174 context-question pairs


In [ ]:
def add_token_positions(encodings, answers):
  start_positions = []
  end_positions = []
  for i in range(len(answers)):
    start_positions.append(encodings.char_to_token(i, answers[i]['answer_start']))
    end_positions.append(encodings.char_to_token(i, answers[i]['answer_end'] - 1))

    # if start position is None, the answer passage has been truncated
    if start_positions[-1] is None:
      start_positions[-1] = tokenizer.model_max_length
    if end_positions[-1] is None:
      end_positions[-1] = tokenizer.model_max_length

  encodings.update({'start_positions': start_positions, 'end_positions': end_positions})

add_token_positions(train_encodings, train_answers)
add_token_positions(valid_encodings, valid_answers)
add_token_positions(test_encodings, test_answers)

In [ ]:
class Dataset(torch.utils.data.Dataset):
  def __init__(self, encodings):
    self.encodings = encodings
  def __getitem__(self, idx):
    return {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
  def __len__(self):
    return len(self.encodings.input_ids)

In [ ]:
train_dataset = Dataset(train_encodings)
valid_dataset = Dataset(valid_encodings)
test_dataset = Dataset(test_encodings)

In [ ]:

from torch.utils.data import DataLoader

# Define the dataloaders
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
valid_loader = DataLoader(valid_dataset, batch_size=16)
test_loader = DataLoader(test_dataset, batch_size=16)

In [ ]:
torch.cuda.empty_cache()

In [ ]:
from transformers import BertForQuestionAnswering, RobertaForQuestionAnswering, DebertaForQuestionAnswering, DebertaV2ForQuestionAnswering

model = BertForQuestionAnswering.from_pretrained("bert-base-uncased")

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForQuestionAnswering: ['cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at bert-base-uncased a

In [ ]:
# Check on the available device - use GPU
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
print(f'Working on {device}')

Working on cuda


In [ ]:
import time

def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [ ]:
import gc
gc.collect()
torch.cuda.empty_cache()

In [ ]:
from transformers import AdamW

N_EPOCHS = 1
LEARNING_RATE = 2e-5
optim = AdamW(model.parameters(), lr=LEARNING_RATE)

model.to(device)
model.train()

for epoch in range(N_EPOCHS):
    epoch_loss = 0
    epoch_acc = 0
    loop = tqdm(train_loader, leave=True)
    for batch in loop:
        optim.zero_grad()
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        start_positions = batch['start_positions'].to(device)
        end_positions = batch['end_positions'].to(device)
        outputs = model(input_ids, attention_mask=attention_mask, start_positions=start_positions, end_positions=end_positions)
        loss = outputs[0]
        loss.backward()
        optim.step()

        epoch_loss += loss.item()

        start_pred = torch.argmax(outputs['start_logits'], dim=1)
        end_pred = torch.argmax(outputs['end_logits'], dim=1)

        acc = ((start_pred == start_positions).sum() + (end_pred == end_positions).sum()) / (2 * len(start_positions))
        epoch_acc += acc.item()

        loop.set_description(f'Epoch {epoch+1}')
        loop.set_postfix(loss=loss.item(), acc=acc.item())

    epoch_loss /= len(train_loader)
    epoch_acc /= len(train_loader)
    print(f'Train Loss:{epoch_loss}, Traning accuracy: {epoch_acc}')

Epoch 1: 100%|██████████| 74/74 [01:42<00:00,  1.38s/it, acc=0.167, loss=5.26]

Train Loss:4.542057256440859, Traning accuracy: 0.13612049556261785


In [ ]:
model_path = '/content/drive/MyDrive/Ajaykumar_NLP/Bert'
model.save_pretrained(model_path)
tokenizer.save_pretrained(model_path)

('/content/drive/MyDrive/Ajaykumar_NLP/Bert/tokenizer_config.json',
 '/content/drive/MyDrive/Ajaykumar_NLP/Bert/special_tokens_map.json',
 '/content/drive/MyDrive/Ajaykumar_NLP/Bert/vocab.txt',
 '/content/drive/MyDrive/Ajaykumar_NLP/Bert/added_tokens.json',
 '/content/drive/MyDrive/Ajaykumar_NLP/Bert/tokenizer.json')

In [ ]:
model_path = '/content/drive/MyDrive/Ajaykumar_NLP/Bert'
model = BertForQuestionAnswering.from_pretrained(model_path)
tokenizer = BertTokenizerFast.from_pretrained(model_path)

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
print(f'Working on {device}')

model = model.to(device)

Working on cuda


In [ ]:
model.eval()

val_loss = 0
val_acc = 0
total_samples = 0

with torch.no_grad():
  for batch in tqdm(valid_loader):
    input_ids = batch['input_ids'].to(device)
    attention_mask = batch['attention_mask'].to(device)
    start_true = batch['start_positions'].to(device)
    end_true = batch['end_positions'].to(device)

    total_samples += input_ids.size(0)
    
    outputs = model(input_ids, attention_mask=attention_mask, start_positions=start_true, end_positions=end_true)

    loss = outputs.loss
    start_pred = torch.argmax(outputs.start_logits, dim=1)
    end_pred = torch.argmax(outputs.end_logits, dim=1)

    val_loss += loss.item() * input_ids.size(0)
    val_acc += ((start_pred == start_true).sum() + (end_pred == end_true).sum()).item()

val_loss /= total_samples
val_acc /= 2 * total_samples

print(f"Validation Loss: {val_loss:.4f}, Validation Accuracy: {val_acc:.4f}")


100%|██████████| 7/7 [00:03<00:00,  2.12it/s]

Validation Loss: 3.4502, Validation Accuracy: 0.2018


In [ ]:
def get_prediction(context, question):
  inputs = tokenizer.encode_plus(question, context, return_tensors='pt', max_length = 512, padding=True, truncation=True, add_special_tokens = True).to(device)
  outputs = model(**inputs)
  
  answer_start = torch.argmax(outputs[0])  
  answer_end = torch.argmax(outputs[1]) + 1 
  
  answer = tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(inputs['input_ids'][0][answer_start:answer_end]))
  
  return answer
  
def question_answer(context, question,answer):
  prediction = get_prediction(context,question)
  return prediction
     

In [ ]:
# BLEU score
import evaluate
bleu = evaluate.load('bleu')
results = 0

for context, question, answer, human_spoiler in zip(test_contexts, test_questions, test_answers, test_human_spoilers):
  if human_spoiler:
    predictions = [f'"{question_answer(context, question, answer).strip()}"']

    if predictions == ['']:
      predictions = ["None"]

    references = [
                  [f'"{human_spoiler.strip()}"'], 
    ]

    results += bleu.compute(predictions=predictions, references=references, max_order = 2).get("bleu")

bleu_score = results/len(test_questions)
print("BLEU Score: ", bleu_score)

BLEU Score:  0.051775537662300614


In [ ]:
# METEOR Score

meteor = evaluate.load('meteor')

results = 0

for context, question, answer, human_spoiler in zip(test_contexts, test_questions, test_answers, test_human_spoilers):
  if human_spoiler:
    predictions = [f'"{question_answer(context, question, answer).strip()}"']

    if predictions == ['']:
      predictions = ["None"]

    references = [
                  f'"{human_spoiler.strip()}"'
    ]
    results += meteor.compute(predictions=predictions, references=references).get("meteor")
    # break

meteor_score = results/len(test_questions)
print("METEOR Score: ", meteor_score)


[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


METEOR Score:  0.3411785080669106


In [ ]:
# BERTScore

from evaluate import load
bertscore = load("bertscore")

results = 0
total_precision = 0
total_recall = 0
total_f1 = 0

count = 0

for context, question, answer, human_spoiler in zip(test_contexts, test_questions, test_answers, test_human_spoilers):
  if human_spoiler:
    predictions = [f'"{question_answer(context, question, answer).strip()}"']

    if predictions == ['']:
      predictions = ["None"]

    references = [
                  f'"{human_spoiler.strip()}"'
    ]
    results = bertscore.compute(predictions=predictions, references=references, lang="en")
    total_precision += results.get("precision")[0]
    total_recall += results.get("recall")[0]
    total_f1 += results.get("f1")[0]
    count += 1

avg_precision = total_precision/len(test_questions)
avg_recall = total_recall/len(test_questions)
avg_f1 = total_f1/len(test_questions)

print("Avg. Precision value: ", avg_precision)
print("Avg. Recall value: ", avg_recall)
print("Avg. F1 value: ", avg_f1)


Avg. Precision value:  0.8436454851235916
Avg. Recall value:  0.8600737310167569
Avg. F1 value:  0.8507080907252298
